# Imports and Environment Variables

In [1]:
network = 'arbitrum' # Can set to variety of networks

In [2]:
from ccip_terminal import (network_func, get_ccip_fee_estimate, send_ccip_transfer, check_ccip_message_status, get_usdc_data)
from web3 import Web3
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
PRIVATE_KEY = os.getenv('PRIVATE_KEY')

In [5]:
w3 = network_func(network)
account = w3.eth.account.from_key(PRIVATE_KEY)
account_obj={"account": account, "w3": w3}

# Sending a CCIP Transfer

In [6]:
to_address = '0x2083b0413869f7b5b9e0eA27d20CB6dD3535f525'
dest_chain='base'
amount=1
source_chain=network

In [ ]:
BALANCES_DICT, TOKEN_CONTRACTS, TOKEN_DECIMALS, _, usdc_price = get_usdc_data(account_obj=account_obj)

❌ Failed to connect to avalanche via https://avax-mainnet.g.alchemy.com/v2/6AUlaGmWe505S7gRPZXVh4YEFgJdYHy5. Trying next...
❌ Failed to connect to avalanche via https://avax-mainnet.g.alchemy.com/v2/6AUlaGmWe505S7gRPZXVh4YEFgJdYHy5. Trying next...


In [12]:
# USDC Balance for Account Across Supported Networks
import json

print(json.dumps(BALANCES_DICT, indent=2))

{
  "0x38979DFdB5d8FD76FAD4E797c4660e20015C6a84": {
    "ethereum": {
      "usdc": 0.0,
      "native_token": 0.0
    },
    "optimism": {
      "usdc": 22.0,
      "native_token": 0.001797209599623622
    },
    "avalanche": {
      "usdc": 1.0,
      "native_token": 0.0
    },
    "arbitrum": {
      "usdc": 1.464262,
      "native_token": 0.010627123766159824
    },
    "polygon": {
      "usdc": 1.0,
      "native_token": 0.0
    },
    "base": {
      "usdc": 5.0,
      "native_token": 0.0
    }
  }
}


In [9]:
TOKEN_CONTRACTS

{'ethereum': '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
 'optimism': '0x0b2C639c533813f4Aa9D7837CAf62653d097Ff85',
 'avalanche': '0xB97EF9Ef8734C71904D8002F8b6Bc66Dd9c48a6E',
 'arbitrum': '0xaf88d065e77c8cC2239327C5EDb3A432268e5831',
 'polygon': '0x3c499c542cEF5E3811e1192ce70d8cC03d5c3359',
 'base': '0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913'}

In [11]:
usdc_price

0.999884

In [20]:
source_chain_state = BALANCES_DICT[account.address][source_chain]
usdc_balance = source_chain_state['usdc']
sufficient_token = amount < usdc_balance

if not sufficient_token:
    print(f'Token balance below desired send amount - Account: {usdc_balance}, Desired amount: {amount}')
else:
    print(f'Token balance is sufficient for send amount')

Token balance is sufficient for send amount


In [ ]:
# We need to use custom fee estimation function for CCIP

estimate = get_ccip_fee_estimate(
    to_address=to_address,
    dest_chain=dest_chain,
    amount=amount,
    source_chain=source_chain,
    w3=w3,
    account=account
)
native_balance = w3.eth.get_balance(account.address)
total_estimate = estimate["total_estimate"]

print(f'native gas balance: {native_balance / 1e18}')
print(f'gas estimate: {total_estimate / 1e18}')

can_transfer = native_balance > total_estimate

print(f'Enough gas for transfer?: {"yes" if can_transfer else "no"}')

native gas balance: 0.010627123766159824
gas estimate: 0.001981512707782155
Enough gas to swap?: yes


In [ ]:
# Note fee is overestimated as we cannot directly estimate gas. For L2 transfers, fee typically ~0.00018 or $0.24

estimate # Denominated in wei

{'total_estimate': 1981512707782155,
 'native_gas_fee': 1803600000000000,
 'ccip_fee': 177912707782155,
 'gas_limit': 360000,
 'max_fee_per_gas': 5010000000,
 'priority_fee': 5000000000,
 'base_gas_price': 10000000,
 'currency': 42161}

In [ ]:
if can_transfer and sufficient_token:
    receipt, links, success, message_id = send_ccip_transfer(
        to_address=to_address,
        dest_chain=dest_chain,
        amount=amount,
        source_chain=source_chain,
        account_obj=account_obj,
        estimate=estimate["total_estimate"]
    )
else:
    missing_gas = estimate["total_estimate"] - native_balance
    missing_token = amount - usdc_balance 
    print(f'Need to top up on gas; need {missing_gas / 1e18} more native token')
    print(f'May need more USDC in account or need to lower desired send amount: {missing_token}')


2025-04-08 18:38:22,687 - INFO - CCIP messageId (pre-send): 46e0dd8ccd971a48ccf2ea84cf297f779b646c7233cc57b661a2009633aa02ac


In [23]:
links

{'source_url': 'https://arbitrum.blockscout.com/tx/0x4ae4aa59f7ea7b1206abea437c155f1f3d1e8ab9e9f6fb6a1541c8320239585d',
 'ccip_url': 'https://ccip.chain.link/#/side-drawer/msg/0x46e0dd8ccd971a48ccf2ea84cf297f779b646c7233cc57b661a2009633aa02ac'}

In [24]:
message_id

'46e0dd8ccd971a48ccf2ea84cf297f779b646c7233cc57b661a2009633aa02ac'

In [26]:
# Here we query Etherscan for transfer status, can take up to 25 minutes.
# Wait flag periodically calls the API for status

check_ccip_message_status(message_id, dest_chain, wait=True)

Attempt 1/15 – retrying in 120s...
Attempt 2/15 – retrying in 120s...
Attempt 3/15 – retrying in 120s...
Attempt 4/15 – retrying in 120s...
Attempt 5/15 – retrying in 120s...
Attempt 6/15 – retrying in 120s...
Attempt 7/15 – retrying in 120s...
Attempt 8/15 – retrying in 120s...
Attempt 9/15 – retrying in 120s...
Attempt 10/15 – retrying in 120s...
Attempt 11/15 – retrying in 120s...
Attempt 12/15 – retrying in 120s...
Found status for message 46e0dd8ccd971a48ccf2ea84cf297f779b646c7233cc57b661a2009633aa02ac on 0x7d38c6363d5e4dfd500a691bc34878b383f58d93: SUCCESS


('SUCCESS', '0x7d38c6363d5e4dfd500a691bc34878b383f58d93')